In [ ]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import wandb
import pandas as pd
from torch import seed
from networkx import selfloop_edges
import csv

Analyze wandb logs

In [ ]:
# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp"  
entity, project = "mjavad", "APD_revised"  

runs = api.runs(entity + "/" + project) 

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains output keys/values for 
    # metrics such as accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
         if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv("project.csv")

In [ ]:
all_runs = []
ids_to_load_train = []
valid_loss_wo = []
valid_loss_w = []
for run in runs:
    if run.state == 'finished':
        if 'TUH_deep4_hps_pp3_wN_woAug_' in run.name: #index2_number2700
            all_runs.append(run)
            valid_loss_wo.append(run.summary['valid_loss'])
        elif 'TUH_deep4_hps_pp3_wN_wAug_' in run.name: #index2_number2700
            all_runs.append(run)
            valid_loss_w.append(run.summary['valid_loss'])

In [ ]:
print(np.mean(valid_loss_w), np.std(valid_loss_w))
print(np.mean(valid_loss_wo),np.std(valid_loss_wo))


In [ ]:
rows = zip(valid_loss_w, valid_loss_wo)#, list3, list4, list5)

# Open a csv file in write mode
with open("data.csv", "w") as f:
    # Create a csv writer object
    writer = csv.writer(f, quoting=csv.QUOTE_ALL)
    writer.writerow(['valid_loss','w=0,wo=1'])
    for row in valid_loss_w:
        writer.writerow([row, '0'])
    for row in valid_loss_wo:
        writer.writerow([row, '1'])

plot loss vs data size

In [ ]:
# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
entity, project = "mjavad", "tuh_scaling"  

runs = api.runs(entity + "/" + project) 

In [ ]:
all_runs = []
ids_to_load_train2 = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
for run in runs:
    if run.state == 'finished':
        if 'NMT_deep4_hps_pp3_wN_wAug_WoPt81_index0' in run.name or 'NMT_deep4_hps_pp3_wN_wAug_WPt81_index9' in run.name: 
            try:
                all_runs.append(run)
                test_loss.append(run.summary['loss_nmt'])
                test_b_acc.append(run.summary['b_acc_nmt'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                ids_to_load_train2.append(run.config["ids_to_load_train2"])
            except:
                print(run.name)
                # run.delete()
                
        
        # elif 'TUH_deep4_hps_pp3_wN_wAug_' in run.name: 
        #     all_runs.append(run)
        #     valid_loss_w.append(run.summary['valid_loss'])

In [ ]:
all_runs = []
ids_to_load_train2 = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        if 'NMT_deep4_hps_pp3_woN_wAug_WoPt81_lr001_index' in run.name or 'NMT_deep4_hps_pp3_woN_wAug_WPt81_lr001_index' in run.name: 
            try:
                all_runs.append(run)
                test_loss.append(run.summary['loss_nmt'])
                test_b_acc.append(run.summary['b_acc_nmt'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                ids_to_load_train2.append(run.config["ids_to_load_train2"])
            except:
                print(run.name)
                # run.delete()
                
        
        # elif 'TUH_deep4_hps_pp3_wN_wAug_' in run.name: 
        #     all_runs.append(run)
        #     valid_loss_w.append(run.summary['valid_loss'])

In [ ]:
# df = pd.DataFrame({'run_name': run_name, 'Pre-Train': pre_train,'ids_to_load_train2': ids_to_load_train2, 'test_loss': test_loss, 'test_b_acc': test_b_acc})
# # df.head()
# df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[:-4].str.join('_'))
# df["ids_to_load_train2"] = df["ids_to_load_train2"].replace(0, 20)
# df.head(10)


In [ ]:

# sns.lineplot(x="ids_to_load_train2", y="test_loss", hue='pre_train', data=df, errorbar="se")
fig = sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='Pre-Train', data=df, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')



import matplotlib.pyplot as plt
ax = plt.gca() # Get the current axes object
ax.set_xscale('log') # Set the y axis scale to log
# plt.xlim([-1,100])
plt.ylabel('Test Balanced Accuracy')
plt.xlabel('Number of Training Samples')

# save the figure as an SVG file
# plt.savefig("../output/pt_Won_tuh_to_nmt.svg", format="svg")


for TUH/NMT random vs fine-tunning

In [ ]:
# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
# entity, project = "mjavad", "tuh_scaling_woN_WoAug_DefArgs" # original 
entity, project = "mjavad", "ft_rnd_nmt"  # 2nd try with 20 min

runs = api.runs(entity + "/" + project) 

In [ ]:
all_runs = []
model = []
ids_to_load_train = []
ids_to_load_train2 = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
seeds = []
for run in runs:
    if run.state == 'finished': # 
        # if 'ft20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # deep4:
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name:# or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_': 
        if 'rnd_nmt_scaling_wN_20min_nAug_DefArgs_' in run.name or 'ft_nmt_scaling_wN_20min_nAug_DefArgs_' in run.name:# or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_':
        # Shallow:
        # if 'ft20_Shallow_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_Shallow_NMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # if 'ft20_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name:
        # EEGnet
        # if 'ft20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name:
        # if 'ft20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_revisionOct_dis09lr' in run.name or 'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name:
        # TCN:
        # if 'ft20_mila_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_wN_WAug_DefArgs_index' in run.name:
        # if 'ft20_mila_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_wN_WAug_DefArgs_index' in run.name:
            try:
                test_b_acc.append(run.summary['b_acc_nmt']) #b_acc_nmt  loss_nmt
                all_runs.append(run)
                test_loss.append(run.summary['loss_tuh'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                model.append(run.config["model_name"])
                ids_to_load_train.append(run.config["ids_to_load_train"])
                ids_to_load_train2.append(run.config["ids_to_load_train2"])
                seeds.append(run.config["seed"])
            except:
                print(run.name)
                # run.delete()
                
        
        # elif 'TUH_deep4_hps_pp3_wN_wAug_' in run.name: 
        #     all_runs.append(run)
        #     valid_loss_w.append(run.summary['valid_loss'])

In [ ]:
df = pd.DataFrame({'run_name': run_name, 'Model':model, 'pre_train': pre_train,'ids_to_load_train': ids_to_load_train,'ids_to_load_train2': ids_to_load_train2, 'test_loss': test_loss, 'test_b_acc': test_b_acc,
                   'seed': seeds})
# df.head()
df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[:-4].str.join('_')) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:
# df = df[df['Model'] == 'Shallow']
# df = df[df['Model'] == 'Deep4Net']
df = df[df['Model'] == 'TCN_var']
# fig = plt.figure(figsize=(16, 8))
# sns.lineplot(x="ids_to_load_train", y="test_loss", hue='Model', data=df, errorbar="se", err_style="bars")
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df, errorbar="se", err_style="band") #, None), orient='y')
fig = sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df,ci=68)#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')

fig.axes.set_yticklabels(fig.axes.get_yticks(), fontsize=24)
fig.axes.set_xticklabels(fig.axes.get_xticks(), fontsize=24)

# define a custom function to format the labels
def format_label(x, pos):
    return f"{x:.2f}"


# create a formatter object using the custom function
formatter = FuncFormatter(format_label)

# set the formatter for the y-axis
fig.axes.yaxis.set_major_formatter(formatter)

ax = plt.gca() # Get the current axes object
ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
# plt.ylim([.48,.84])
# plt.ylim([.60,.80])

# plt.xlim([30,2800])

# plt.ylabel('Test Balanced Accuracy')
# plt.xlabel('Number of Training Samples')

# plt.ylabel(None)
# plt.xlabel(None)

# plt.gca().legend().remove()

# plt.savefig("../output/eegnet_pos_trans_oct2023.svg", format="svg")
# plt.savefig("../output/deep4_forget_oct2023.svg", format="svg")


In [ ]:
# df

In [ ]:
# df[df["ids_to_load_train2"] >1000]

In [ ]:
# sns.bo

In [ ]:
# df.groupby(['Model', 'ids_to_load_train2']).count()

In [ ]:
# df[(df['ids_to_load_train2']>2000) & (df['pre_train']==False) ]

In [ ]:
stop  # #########################################################################################

For single vs concatinated training set (table)

In [ ]:

# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
entity, project = "mjavad", "tuh_scaling_woN_WoAug_DefArgs"  

runs = api.runs(entity + "/" + project) 

In [ ]:
all_runs = []
model = []
ids_to_load_train = []
ids_to_load_train2 = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_'  in run.name : 
        # if 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_'  in run.name : 
        # if 'ft20_Shallow_NMT_scaling_wN_WAug_DefArgs_'  in run.name : 
        if 'ft20_EEGNet_NMT_scaling_wN_WAug_DefArgs_'  in run.name : 
        # if 'tuh_scaling_wN_WAug_DefArgs' in run.name :
        # if 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if 'rnd5_mila_Allmodels_merge_full_wN_WAug_DefArgs_' in run.name:
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # if 'ft20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name:
            try:
                test_b_acc.append(run.summary['b_acc_merge'])
                all_runs.append(run)
                test_loss.append(run.summary['loss_tuh'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                model.append(run.config["model_name"])
                ids_to_load_train.append(run.config["ids_to_load_train"])
                ids_to_load_train2.append(run.config["ids_to_load_train2"])
            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({'run_name': run_name, 'Model':model, 'pre_train': pre_train,'ids_to_load_train': ids_to_load_train,'ids_to_load_train2': ids_to_load_train2, 'test_loss': test_loss, 'test_b_acc': test_b_acc})
# df.head()
# df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[:-4].str.join('_')) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:
df = df[df['ids_to_load_train2'] == 2110]
df.groupby(['Model']).std()


Discriminative learning rates

In [ ]:

# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
entity, project = "mjavad", "tuh_scaling_woN_WoAug_DefArgs" 
# entity, project = "mjavad", "APD_revised"  


runs = api.runs(entity + "/" + project) 

In [ ]:
all_runs = []
model = []
ids_to_load_train = []
ids_to_load_train2 = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        # if 'tuh_scaling_wN_WAug_DefArgs' in run.name :
        # if 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        if 'ftNormLr_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name or 'ftDisLr_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name:
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # if 'ft20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name:
            try:
                test_b_acc.append(run.summary['b_acc_nmt'])
                all_runs.append(run)
                test_loss.append(run.summary['loss_tuh'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                model.append(run.config["model_name"])
                ids_to_load_train.append(run.config["ids_to_load_train"])
                ids_to_load_train2.append(run.config["ids_to_load_train2"])
            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({'run_name': run_name, 'Model':model, 'pre_train': pre_train,'ids_to_load_train': ids_to_load_train,'ids_to_load_train2': ids_to_load_train2, 'test_loss': test_loss, 'test_b_acc': test_b_acc})
# df.head()
df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[0]) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:


# fig = plt.figure(figsize=(16, 8))
# sns.lineplot(x="ids_to_load_train", y="test_loss", hue='Model', data=df, errorbar="se", err_style="bars")
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df, errorbar="se", err_style="band") #, None), orient='y')
sns.boxplot(x="run_name_tranc", y="test_b_acc", data=df, order=['ftNormLr','ftDisLr', 'ftDisLr05'])#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')

# ax = plt.gca() # Get the current axes object
# ax.set_xscale('log') # Set the y axis scale to log
# # ax.set_yscale('log') # Set the y axis scale to log
# # plt.xlim([-1,100])
plt.ylabel('Test Balanced Accuracy')
plt.xlabel('Discriminative Learning Rates')

# plt.savefig("../output/Deep4_dis_LR.svg", format="svg")


In [ ]:
df.groupby('run_name_tranc').mean()

In [ ]:
df.to_csv('data.csv', index=False)

Scaling model and data size

In [ ]:

# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
# entity, project = "mjavad", "tuh_scaling_woN_WoAug_DefArgs" 
entity, project = "mjavad", "tuh_nFilters" # #"tcn_small15" #APD_revised_CpLoss #tuh_nFilters


runs = api.runs(entity + "/" + project) 

In [ ]:


all_runs = []
model = []
ids_to_load_train = []
n_tcn_blocks = []
n_tcn_filters = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
augment = []
seed = []
for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        # if 'tuh_scaling_wN_WAug_DefArgs' in run.name :
        # if 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if 'tuh_size_scale_model_' in run.name:# or 'ftDisLr_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name:
        # if 'nmt_nAaug_size_scale_model_' in run.name or 'nmt_wAaug_size_scale_model_' in run.name:# or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name:        
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # if 'ft20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        if 'tuh_wN_nAug_model_TCN_var_' in run.name: #or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name:
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name:
            try:
                test_b_acc.append(run.summary['b_acc_tuh']) #loss_nmt b_acc_nmt
                all_runs.append(run)
                test_loss.append(run.summary['loss_nmt'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                model.append(run.config["model_name"])
                ids_to_load_train.append(run.config["ids_to_load_train"])
                n_tcn_blocks.append(run.config["n_tcn_blocks"])
                n_tcn_filters.append(run.config["n_tcn_filters"])
                seed.append(run.config["seed"])
                augment.append(run.config["augment"])
            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({'run_name': run_name, 'Model':model, 'pre_train': pre_train,'ids_to_load_train': ids_to_load_train,
                   'n_tcn_blocks': n_tcn_blocks, 'n_tcn_filters': n_tcn_filters,
                    'test_loss': test_loss, 'test_b_acc': test_b_acc,
                   'seed': seed, 'augment': augment})
# df.head()
# df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[2]) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:


# df = df[df['augment'] == True]
# df = df[df['n_tcn_blocks'].isin([6])]
# df = df[df['seed'].isin([2023])]
# fig = plt.figure(figsize=(16, 8))
# sns.lineplot(x="ids_to_load_train", y="test_loss", hue='Model', data=df, errorbar="se", err_style="bars")
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df, errorbar="se", err_style="band") #, None), orient='y')
# sns.boxplot(x="run_name_tranc", y="test_b_acc", data=df, order=['ftNormLr','ftDisLr', 'ftDisLr05'])#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train", y="test_loss", data=df, hue="n_tcn_blocks")#, errorbar="se", err_style="bars") #, None), orient='y')
sns.lineplot(x="ids_to_load_train", y="test_b_acc", data=df, hue="n_tcn_filters")#, errorbar="se", err_style="bars") #, None), orient='y')
# n_tcn_blocks

ax = plt.gca() # Get the current axes object
ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
# # plt.xlim([-1,100])
plt.ylabel('Test Balanced Accuracy')
plt.xlabel('Number of Training Samples')

# plt.savefig("../output/Deep4_dis_LR.svg", format="svg")


Ablation: compare different pretraining methods

In [ ]:

# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
# entity, project = "mjavad", "tuh_scaling_woN_WoAug_DefArgs" 
entity, project = "mjavad", "tuh_ablation_pp" # #"tcn_small15" #APD_revised_CpLoss #tuh_nFilters


runs = api.runs(entity + "/" + project) 

In [ ]:


all_runs = []
model = []
ids_to_load_train = []
n_tcn_blocks = []
n_tcn_filters = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
augment = []
seed = []
for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        # if 'tuh_scaling_wN_WAug_DefArgs' in run.name :
        # if 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if 'tuh_size_scale_model_' in run.name:# or 'ftDisLr_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name:
        # if 'nmt_nAaug_size_scale_model_' in run.name or 'nmt_wAaug_size_scale_model_' in run.name:# or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name:        
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # if 'ft20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        if 'tuh_nzscore20min_n01_4B128_valNMT_model_' in run.name or 'tuh_nzscore5min_n01_4B128_valNMT_model_' in run.name or 'tuh_wzscore20min_n01_4B128_valNMT_model_' in run.name or 'tuh_wzscore5min_n01_4B128_valNMT_model_' in run.name:
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name:
            try:
                test_b_acc.append(run.summary['b_acc_nmt']) #loss_nmt b_acc_nmt
                all_runs.append(run)
                test_loss.append(run.summary['loss_nmt'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                model.append(run.config["model_name"])
                ids_to_load_train.append(run.config["ids_to_load_train"])
                n_tcn_blocks.append(run.config["n_tcn_blocks"])
                n_tcn_filters.append(run.config["n_tcn_filters"])
                seed.append(run.config["seed"])
                augment.append(run.config["augment"])
            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({'run_name': run_name, 'Model':model, 'pre_train': pre_train,'ids_to_load_train': ids_to_load_train,
                   'n_tcn_blocks': n_tcn_blocks, 'n_tcn_filters': n_tcn_filters,
                    'test_loss': test_loss, 'test_b_acc': test_b_acc,
                   'seed': seed, 'augment': augment})
# df.head()
df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[1]) #+ '_' + str(x.pre_train))

df = df.assign(zscore_status = lambda x: x.run_name_tranc.str[0]) #+ '_' + str(x.pre_train))
df = df.assign(min_status = lambda x: x.run_name_tranc.str[-4:-3]) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:


# df = df[df['augment'] == True]
# df = df[df['n_tcn_blocks'].isin([6])]
# df = df[df['seed'].isin([2023])]
# fig = plt.figure(figsize=(16, 8))
# sns.lineplot(x="ids_to_load_train", y="test_loss", hue='Model', data=df, errorbar="se", err_style="bars")
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df, errorbar="se", err_style="band") #, None), orient='y')
fig = sns.boxplot(x="min_status", y="test_b_acc", hue="zscore_status", data=df, order=["5","0"], hue_order=["n","w"])#, order=['nzscore5min','wzscore5min','nzscore20min', 'wzscore20min'])#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train", y="test_loss", data=df, hue="n_tcn_blocks")#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train", y="test_b_acc", data=df, hue="n_tcn_filters")#, errorbar="se", err_style="bars") #, None), orient='y')
# n_tcn_blocks

ax = plt.gca() # Get the current axes object
# ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
# # plt.xlim([-1,100])
# plt.ylabel('Test Balanced Accuracy')
# plt.xlabel('Number of Training Samples')

# Change the y-axis font size to 24
# Get the 2D array of Axes objects
# axes = 
fig.axes.set_yticklabels(ax.get_yticks(), fontsize=24)

# define a custom function to format the labels
def format_label(x, pos):
    return f"{x:.2f}"


# create a formatter object using the custom function
formatter = FuncFormatter(format_label)

# set the formatter for the y-axis
fig.axes.yaxis.set_major_formatter(formatter)

plt.savefig("../output/Abl_oodNMT_wnzscore_5_20min.svg", format="svg")
# plt.savefig("../output/Abl_iidTUAB_wnzscore_5_20min.svg", format="svg")


In [ ]:
sdcfe

well known Models scaling with data size

In [ ]:
# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
entity, project = "mjavad", "tuh_scaling_woN_WoAug_DefArgs"  

runs = api.runs(entity + "/" + project) 

In [ ]:
all_runs = []
model = []
ids_to_load_train = []
ids_to_load_train2 = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_'  in run.name : 
        # if 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_'  in run.name : 
        # if 'ft20_Shallow_NMT_scaling_wN_WAug_DefArgs_'  in run.name : 
        # if 'ft20_EEGNet_NMT_scaling_wN_WAug_DefArgs_'  in run.name : 
        # fig_scaling onn the paper
        # if 'tuh_scaling_wN_WAug_DefArgs' in run.name :
        # revision scaling with 20 min data
        if 'tuh_scaling_wN_20min_nAug_DefArgs_' in run.name :
        # if 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if 'rnd5_mila_Allmodels_merge_full_wN_WAug_DefArgs_' in run.name:
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # if 'ft20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name:
            try:
                test_b_acc.append(run.summary['b_acc_nmt']) #b_acc_nmt
                all_runs.append(run)
                test_loss.append(run.summary['loss_tuh'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                model.append(run.config["model_name"])
                ids_to_load_train.append(run.config["ids_to_load_train"])
                ids_to_load_train2.append(run.config["ids_to_load_train2"])
            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({'run_name': run_name, 'Model':model, 'pre_train': pre_train,'ids_to_load_train': ids_to_load_train,'ids_to_load_train2': ids_to_load_train2, 'test_loss': test_loss, 'test_b_acc': test_b_acc})
# df.head()
# df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[:-4].str.join('_')) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:
df_org = df.copy()
# df = df[df['augment'] == True]
# df = df[df['n_tcn_blocks'].isin([1])]
# df = df[df['n_tcn_filters']< 128]
# df = df[df['n_tcn_filters']==64]
# df = df[df['n_tcn_filters'].isin([4,64])]


# df = df[df['seed'].isin([2023])]
# fig = plt.figure(figsize=(16, 8))
# sns.lineplot(x="ids_to_load_train", y="test_loss", hue='Model', data=df, errorbar="se", err_style="bars")
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df, errorbar="se", err_style="band") #, None), orient='y')
# sns.boxplot(x="run_name_tranc", y="test_b_acc", data=df, order=['nzscore5min','wzscore5min','nzscore20min', 'wzscore20min'])#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train", y="test_loss", data=df, hue="n_tcn_blocks")#, errorbar="se", err_style="bars") #, None), orient='y')
fig = sns.lineplot(x="ids_to_load_train", y="test_b_acc", data=df, hue="Model", ci=68, hue_order=["EEGNet", "Shallow", "Deep4Net", "TCN"])#, err_style="bars") #, None), orient='y') , err_style="bars"
# n_tcn_blocks

# Plot the original data
# sns.lineplot(x="ids_to_load_train", y="test_b_acc", data=df, hue="n_tcn_filters", ci=68)

# Apply a 10 percent smoothing using gaussian_filter1d
# from scipy.ndimage import gaussian_filter1d
# y_smooth = gaussian_filter1d(df["test_b_acc"], sigma=0.01 * df.shape[0])
# sns.lineplot(x="ids_to_load_train", y=y_smooth, data=df, hue="n_tcn_filters")



fig.axes.set_yticklabels(fig.axes.get_yticks(), fontsize=24)
fig.axes.set_xticklabels(fig.axes.get_xticks(), fontsize=24)

# define a custom function to format the labels
def format_label(x, pos):
    return f"{x:.2f}"


# create a formatter object using the custom function
formatter = FuncFormatter(format_label)

# set the formatter for the y-axis
fig.axes.yaxis.set_major_formatter(formatter)

ax = plt.gca() # Get the current axes object
ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
# plt.ylim([.48,.83])
plt.ylim([.48,.713])
# plt.xlim([200, 600])

# plt.ylabel('Test Balanced Accuracy')
# plt.xlabel('Number of Training Samples')

plt.ylabel(None)
plt.xlabel(None)

# plt.gca().legend().remove()

# plt.savefig("../output/all_wellKnown_oodNMT.svg", format="svg")


In [ ]:
len(df)
df2 = df[df['ids_to_load_train'] == 31]
df3 = df2[df2['Model'] == 'Deep4Net']
df3

In [ ]:
stop 

Number of TNC filters effect 

In [ ]:

# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
# entity, project = "mjavad", "tuh_scaling_woN_WoAug_DefArgs" 
entity, project = "mjavad", "tuh_ablation_nf_clean" # #"tcn_small15" #APD_revised_CpLoss #tuh_nFilters


runs = api.runs(entity + "/" + project) 

In [ ]:


all_runs = []
model = []
ids_to_load_train = []
n_tcn_blocks = []
n_tcn_filters = []
test_loss = []
run_name = []
pre_train = []
test_b_acc =[]
augment = []
seed = []
for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        # if 'tuh_scaling_wN_WAug_DefArgs' in run.name :
        # if 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if  'rnd20_mila_EEGNet_NMT_scaling_wN_WAug_DefArgs_' in run.name :
        # if 'tuh_size_scale_model_' in run.name:# or 'ftDisLr_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name:
        # if 'nmt_nAaug_size_scale_model_' in run.name or 'nmt_wAaug_size_scale_model_' in run.name:# or 'ftDisLr05_rnd5_mila_Deep4Net_Merge_scaling_wN_WAug_DefArgs_' in run.name:        
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name: 
        # if 'ft20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_TCN_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'ft20_deep4_NMT_scaling_wN_WAug_DefArgs_' in run.name or 'rnd20_RndNMT_scaling_wN_WAug_DefArgs_' in run.name or 'ft20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_Deep4Net_NMT_scaling_woN_WAug_DefArgs_index' in run.name: 
        # if 'tuh_wzscore20min_n01_nBn_model_' in run.name:# or 'tuh_nzscore5min_n01_4B128_valNMT_model_' in run.name or 'tuh_wzscore20min_n01_4B128_valNMT_model_' in run.name or 'tuh_wzscore5min_n01_4B128_valNMT_model_' in run.name:
        if 'tuh_wzscore20min_n01_nBlocks_nBn_model_' in run.name or 'tuh_wzscore20min_n01_nBn_model_' in run.name:# or 'tuh_wzscore20min_n01_4B128_valNMT_model_' in run.name or 'tuh_wzscore5min_n01_4B128_valNMT_model_' in run.name:
        # if 'ft20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_' in run.name or 'rnd20_mila_TCN_NMT_scaling_woN_WAug_DefArgs_index' in run.name:
            try:
                test_b_acc.append(run.summary['loss_nmt']) #loss_nmt b_acc_nmt
                all_runs.append(run)
                test_loss.append(run.summary['loss_nmt'])
                run_name.append(run.name)
                pre_train.append(run.config["pre_trained"])
                model.append(run.config["model_name"])
                ids_to_load_train.append(run.config["ids_to_load_train"])
                n_tcn_blocks.append(run.config["n_tcn_blocks"])
                n_tcn_filters.append(run.config["n_tcn_filters"])
                seed.append(run.config["seed"])
                augment.append(run.config["augment"])
            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({'run_name': run_name, 'Model':model, 'pre_train': pre_train,'ids_to_load_train': ids_to_load_train,
                   'n_tcn_blocks': n_tcn_blocks, 'n_tcn_filters': n_tcn_filters,
                    'test_loss': test_loss, 'test_b_acc': test_b_acc,
                   'seed': seed, 'augment': augment})
# df.head()
df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[1]) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:


df_org = df.copy()
# df = df[df['augment'] == True]
df = df[df['n_tcn_blocks'].isin([1])]
df = df[df['n_tcn_filters']< 128]
# df = df[df['n_tcn_filters']==64]
# df = df[df['n_tcn_filters'].isin([4,64])]


# df = df[df['seed'].isin([2023])]
# fig = plt.figure(figsize=(16, 8))
# sns.lineplot(x="ids_to_load_train", y="test_loss", hue='Model', data=df, errorbar="se", err_style="bars")
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df, errorbar="se", err_style="band") #, None), orient='y')
# sns.boxplot(x="run_name_tranc", y="test_b_acc", data=df, order=['nzscore5min','wzscore5min','nzscore20min', 'wzscore20min'])#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train", y="test_loss", data=df, hue="n_tcn_blocks")#, errorbar="se", err_style="bars") #, None), orient='y')
fig = sns.lineplot(x="ids_to_load_train", y="test_b_acc", data=df, hue="n_tcn_filters", ci=68)#, err_style="bars") #, None), orient='y')
# n_tcn_blocks

# Plot the original data
# sns.lineplot(x="ids_to_load_train", y="test_b_acc", data=df, hue="n_tcn_filters", ci=68)

# Apply a 10 percent smoothing using gaussian_filter1d
# from scipy.ndimage import gaussian_filter1d
# y_smooth = gaussian_filter1d(df["test_b_acc"], sigma=0.01 * df.shape[0])
# sns.lineplot(x="ids_to_load_train", y=y_smooth, data=df, hue="n_tcn_filters")



fig.axes.set_yticklabels(fig.axes.get_yticks(), fontsize=24)
fig.axes.set_xticklabels(fig.axes.get_xticks(), fontsize=24)

# define a custom function to format the labels
def format_label(x, pos):
    return f"{x:.2f}"


# create a formatter object using the custom function
formatter = FuncFormatter(format_label)

# set the formatter for the y-axis
fig.axes.yaxis.set_major_formatter(formatter)

ax = plt.gca() # Get the current axes object
ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
# plt.ylim([.48,.71])
# plt.ylabel('Test Balanced Accuracy')
# plt.xlabel('Number of Training Samples')

plt.ylabel(None)
plt.xlabel(None)

# plt.gca().legend().remove()

# plt.savefig("../output/tcn_nf_oodNMT.svg", format="svg")


plot number of TNC blocks

In [ ]:
df = df_org.copy()



# df_org = df.copy()
# df = df[df['augment'] == True]
# df = df[df['n_tcn_blocks'].isin([1,4])]
# df = df[df['n_tcn_filters']< 128]
df = df[df['n_tcn_filters']==8]

# df = df[df['seed'].isin([2023])]
# fig = plt.figure(figsize=(16, 8))
# sns.lineplot(x="ids_to_load_train", y="test_loss", hue='Model', data=df, errorbar="se", err_style="bars")
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='pre_train', data=df, errorbar="se", err_style="band") #, None), orient='y')
# sns.boxplot(x="run_name_tranc", y="test_b_acc", data=df, order=['nzscore5min','wzscore5min','nzscore20min', 'wzscore20min'])#, errorbar="se", err_style="bars") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train2", y="test_b_acc", hue='run_name_tranc', data=df, errorbar="se") #, None), orient='y')
# sns.lineplot(x="ids_to_load_train", y="test_loss", data=df, hue="n_tcn_blocks")#, errorbar="se", err_style="bars") #, None), orient='y')
fig=sns.lineplot(x="ids_to_load_train", y="test_b_acc", data=df, hue="n_tcn_blocks", ci=68)#, errorbar="se", err_style="bars") #, None), orient='y')
# n_tcn_blocks




fig.axes.set_yticklabels(fig.axes.get_yticks(), fontsize=24)
fig.axes.set_xticklabels(fig.axes.get_xticks(), fontsize=24)

# define a custom function to format the labels
def format_label(x, pos):
    return f"{x:.2f}"


# create a formatter object using the custom function
formatter = FuncFormatter(format_label)

# set the formatter for the y-axis
fig.axes.yaxis.set_major_formatter(formatter)

ax = plt.gca() # Get the current axes object
ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
# plt.ylim([.48,.83])
# plt.ylabel('Test Balanced Accuracy')
# plt.xlabel('Number of Training Samples')

plt.ylabel(None)
plt.xlabel(None)

plt.gca().legend().remove()

# plt.savefig("../output/tcn_nb_iidtuh.svg", format="svg")


In [ ]:
# df[df["n_tcn_blocks"]==6]


In [ ]:
done

Sex clf results

In [ ]:
# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
entity, project = "mjavad", "EEG_sex_narval"  

runs = api.runs(entity + "/" + project) 

In [ ]:
all_runs = []
model = []
test_set_nmt_normal_male = []
test_set_nmt_abnormal_male = []
test_set_nmt_normal_female = []
test_set_nmt_abnormal_female = []

for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        if 'NMT_test_subgroups_wZscore5min_w11_wAug_model_' in run.name: # or 'TUH_wZscore5min_w11_wAug_model_' in run.name:# or 'tuh_wzscore20min_n01_4B128_valNMT_model_' in run.name or 'tuh_wzscore5min_n01_4B128_valNMT_model_' in run.name:

            try:
                test_set_nmt_normal_male.append(run.summary['test_set_nmt_normal_male']) 
                test_set_nmt_abnormal_male.append(run.summary['test_set_nmt_abnormal_male']) 
                test_set_nmt_normal_female.append(run.summary['test_set_nmt_normal_female']) 
                test_set_nmt_abnormal_female.append(run.summary['test_set_nmt_abnormal_female']) 
                
            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({'test_set_nmt_normal_male':test_set_nmt_normal_male,
                   'test_set_nmt_abnormal_male':test_set_nmt_abnormal_male,
                   'test_set_nmt_normal_female':test_set_nmt_normal_female,
                   'test_set_nmt_abnormal_female':test_set_nmt_abnormal_female
                   })
# df.head()
# df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[1]) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)


In [ ]:
import pandas as pd
import seaborn as sns

# Assuming df is your DataFrame and the columns are as you mentioned
df_melted = df.melt( value_vars=['test_set_nmt_normal_male', 'test_set_nmt_abnormal_male', 'test_set_nmt_normal_female', 'test_set_nmt_abnormal_female'], var_name='Category', value_name='test_acc')

# # Extract gender and condition from the Category
df_melted['Gender'] = df_melted['Category'].apply(lambda x: 'Female' if 'female' in x else 'Male')
df_melted['Condition'] = df_melted['Category'].apply(lambda x: 'Abnormal' if 'abnormal' in x else 'Normal')

# # Now plot
fig = sns.boxplot(x='Gender', y='test_acc', data=df_melted, hue='Condition', order=['Female', 'Male'], hue_order=['Abnormal', 'Normal'])



fig.axes.set_yticklabels(fig.axes.get_yticks(), fontsize=24)
# fig.axes.set_xticklabels(fig.axes.get_xticks(), fontsize=24)

# define a custom function to format the labels
def format_label(x, pos):
    return f"{x:.2f}"


# create a formatter object using the custom function
formatter = FuncFormatter(format_label)

# set the formatter for the y-axis
fig.axes.yaxis.set_major_formatter(formatter)

ax = plt.gca() # Get the current axes object
# ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
plt.ylim([.48,.93])
# plt.ylabel('Test Balanced Accuracy')
# plt.xlabel('Number of Training Samples')

plt.ylabel(None)
plt.xlabel(None)

plt.gca().legend().remove()

# plt.savefig("../output/sex_imbalance_MF_NA_shallow_iidnmt.svg", format="svg")


Sex detection results

In [ ]:
# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
entity, project = "mjavad", "EEG_sex_report_final"  

runs = api.runs(entity + "/" + project) 

In [ ]:
from numpy import NaN


all_runs = []
model = []

run_name = []

b_acc_tuab = []
b_acc_nmt = []
b_acc_tueg = []

ids_to_load_train = []
ids_to_load_train2 = []
ids_to_load_train3 = []

for run in runs:
    if run.state == 'finished': # NMT_deep4_hps_pp3_wN_wAug_WoPt81_28jul2045_index
        if 'w11' in run.name: # or 'TUH_wZscore5min_w11_wAug_model_' in run.name:# or 'tuh_wzscore20min_n01_4B128_valNMT_model_' in run.name or 'tuh_wzscore5min_n01_4B128_valNMT_model_' in run.name:

            try:
                
                if 'TUEGonlyNormal' in run.name or 'TUEGonlyAbNormal' in run.name:
                    b_acc_tuab.append(NaN) 
                    b_acc_nmt.append(NaN) 
                    b_acc_tueg.append(NaN)
                else:
                    b_acc_tuab.append(run.summary['b_acc_tuh']) 
                    b_acc_nmt.append(run.summary['b_acc_nmt']) 
                    b_acc_tueg.append(run.summary['b_acc_tueg']) 

                ids_to_load_train.append(run.config["ids_to_load_train"])
                ids_to_load_train2.append(run.config["ids_to_load_train2"])
                ids_to_load_train3.append(run.config["ids_to_load_train3"])

                run_name.append(run.name) 

                

            except:
                print(run.name)
                # run.delete()
                
df = pd.DataFrame({
    'run_name':run_name,
    'b_acc_tuab':b_acc_tuab,
    'b_acc_nmt':b_acc_nmt,
    'b_acc_tueg':b_acc_tueg,
    'ids_to_load_train':ids_to_load_train,
    'ids_to_load_train2':ids_to_load_train2,
    'ids_to_load_train3':ids_to_load_train3       
    })
# df.head()
df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[0]) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)
# df


In [ ]:
import pandas as pd
import seaborn as sns
sns.set_palette("colorblind")

# Assuming df is your DataFrame and the columns are as you mentioned
df_melted = df.melt(
    id_vars=['run_name_tranc'],
                     value_vars=['b_acc_tuab', 'b_acc_nmt', 'b_acc_tueg'],
                       var_name='Category', value_name='test_acc')
# df_melted = df_melted.melt(
#             id_vars=['test_acc','Category'],
#             value_vars=['ids_to_load_train', 'ids_to_load_train2', 'ids_to_load_train3'],
#             var_name='ids_to_load_train', value_name='ids_to_load_train_name')
# print(df_melted.head())

# # Extract gender and condition from the Category
# df_melted['Gender'] = df_melted['Category'].apply(lambda x: 'Female' if 'female' in x else 'Male')
# df_melted['Condition'] = df_melted['Category'].apply(lambda x: 'Abnormal' if 'abnormal' in x else 'Normal')

# # Now plot
fig = sns.barplot(x='run_name_tranc', y='test_acc', data=df_melted, hue='Category',
                  ci=68,
                #    order=['TUHonlyNormal', 'NMTonlyNormal', 'TUEGonlyNormal'],
                #   order=['TUHonlyAbNormal', 'NMTonlyAbNormal', 'TUEGonlyAbNormal'],
                   order = ['TUHall','NMTall','TUEGall'],
                 hue_order=['b_acc_tuab', 'b_acc_nmt', 'b_acc_tueg']
                 
                )



fig.axes.set_yticklabels(fig.axes.get_yticks(), fontsize=24)
# fig.axes.set_xticklabels(fig.axes.get_xticks(), fontsize=24)
# fig.axes.set_xticklabels(None, fontsize=24)

# define a custom function to format the labels
def format_label(x, pos):
    return f"{x:.2f}"


# create a formatter object using the custom function
formatter = FuncFormatter(format_label)

# set the formatter for the y-axis
fig.axes.yaxis.set_major_formatter(formatter)

ax = plt.gca() # Get the current axes object
# ax.set_xscale('log') # Set the y axis scale to log
# ax.set_yscale('log') # Set the y axis scale to log
plt.ylim([.48,.83])
# plt.ylabel('Test Balanced Accuracy')
# plt.xlabel('Number of Training Samples')

plt.ylabel(None)
plt.xlabel(None)
fig.xaxis.set_ticklabels([])

# plt.gca().legend().remove()

# plt.savefig("../output/sex_onlyNormal_3ds_cb.svg", format="svg")
# plt.savefig("../output/sex_onlyAbNormal_3ds_cb.svg", format="svg")
# plt.savefig("../output/sex_all_3ds_cb_legends.svg", format="svg")


Sex detectability with more seedsusing wandb sweep

In [ ]:
# Run wandb login on the command line and paste in your API key
api = wandb.Api()
# entity, project = "mjavad", "tuh_sc_hps_pp3_cp_wN"  
entity, project, sweep_name = "mjavad", "EEG_sex" ,  "u4826d3c"

runs = api.runs(entity + "/" + project)

In [ ]:
from numpy import NaN


all_runs = []
model = []

run_name = []

b_acc_tuab = []
b_acc_nmt = []
b_acc_tueg = []

ids_to_load_train = []
ids_to_load_train2 = []
ids_to_load_train3 = []
train_ons = []
pathology_split = []

for run in runs:
    if run.sweep is None or run.sweep.id != sweep_name:
        continue
    if run.state != "finished": # or run.config['r'] != r:
        continue

    # try:       
    b_acc_tuab.append(run.summary['b_acc_tuh']) 
    b_acc_nmt.append(run.summary['b_acc_nmt']) 
    b_acc_tueg.append(run.summary['b_acc_tueg']) 
    

    ids_to_load_train.append(run.config["ids_to_load_train"])
    ids_to_load_train2.append(run.config["ids_to_load_train2"])
    ids_to_load_train3.append(run.config["ids_to_load_train3"])
    pathology_split.append(run.config["pathology_split"])
    train_ons.append(run.config["train_on"])

    run_name.append(run.name) 

        # b_acc_tueg.append(run.summary['b_acc_tueg']) 


        

    # except:
    #     print(run.name)
    #     # run.delete()
        
df = pd.DataFrame({
    'run_name':run_name,
    'b_acc_tuab':b_acc_tuab,
    'b_acc_nmt':b_acc_nmt,
    'b_acc_tueg':b_acc_tueg,
    'ids_to_load_train':ids_to_load_train,
    'ids_to_load_train2':ids_to_load_train2,
    'ids_to_load_train3':ids_to_load_train3,
    'train_ons':train_ons,
    'pathology_split':  pathology_split    
    })
# df.head()
# df = df.assign(run_name_tranc = lambda x: x.run_name.str.split('_').str[0]) #+ '_' + str(x.pre_train))

# import numpy as np
# df["run_name_tranc_pretrain"] = df["run_name_tranc"] + "_" + np.where(df["pre_train"], "true", "false")

# df = df.assign(run_name_tranc = lambda x: str(x.pre_train))
# df["ids_to_load_train"] = df["ids_to_load_train"].replace(0, 20)
df.head(5)
# df



In [ ]:
from calendar import c
import pandas as pd
import seaborn as sns
sns.set_palette("colorblind")

for cond in ['normal', 'abnormal', 'all']:

  # Assuming df is your DataFrame and the columns are as you mentioned
  df_melted = df[(df['pathology_split'] == cond)].melt(
      id_vars=['train_ons'],
                      value_vars=['b_acc_tuab', 'b_acc_nmt', 'b_acc_tueg'],
                        var_name='Category', value_name='test_acc')
  # # Now plot
  fig = sns.barplot(x='train_ons', y='test_acc', data=df_melted, hue='Category',
                    errorbar=('ci', 68),
                  #    order=['TUHonlyNormal', 'NMTonlyNormal', 'TUEGonlyNormal'],
                  #   order=['TUHonlyAbNormal', 'NMTonlyAbNormal', 'TUEGonlyAbNormal'],
                    #  order = ['TUHall','NMTall','TUEGall'],
                    order = ['only_TUAB','only_NMT', 'only_TUEG'],
                  hue_order=['b_acc_tuab', 'b_acc_nmt', 'b_acc_tueg']
                  
                  )

  fig.axes.set_yticklabels(fig.axes.get_yticks(), fontsize=24)
  # fig.axes.set_xticklabels(fig.axes.get_xticks(), fontsize=24)
  # fig.axes.set_xticklabels(None, fontsize=24)

  # define a custom function to format the labels
  def format_label(x, pos):
      return f"{x:.2f}"


  # create a formatter object using the custom function
  formatter = FuncFormatter(format_label)

  # set the formatter for the y-axis
  fig.axes.yaxis.set_major_formatter(formatter)

  ax = plt.gca() # Get the current axes object
  # ax.set_xscale('log') # Set the y axis scale to log
  # ax.set_yscale('log') # Set the y axis scale to log
  plt.ylim([.48,.83])
  # plt.ylabel('Test Balanced Accuracy')
  # plt.xlabel('Number of Training Samples')

  plt.ylabel(None)
  plt.xlabel(None)
  fig.xaxis.set_ticklabels([])

  plt.show()
  # plt.gca().legend().remove()

  # plt.savefig("../output/sex_onlyNormal_3ds_cb.svg", format="svg")
  # plt.savefig("../output/sex_onlyAbNormal_3ds_cb.svg", format="svg")
  # plt.savefig("../output/sex_all_3ds_cb_legends.svg", format="svg")


In [ ]:
stop


Appendix
:P

In [ ]:
df.groupby(['Model', 'ids_to_load_train2']).count()

In [ ]:
df[df['Model']=='Deep4Net'].groupby(['ids_to_load_train']).std().sort_values(by='test_b_acc', ascending=False).head(20)
# df[df['Model']=='TCN'].sort_values(by='test_b_acc', ascending=False).head(20)

In [ ]:
df[df['Model']=='TCN'].sort_values(by='test_b_acc', ascending=False).head(20)

In [ ]:
# run = runs[-2]
df[df["ids_to_load_train2"]==0]
df.plot(x="ids_to_load_train2", y="test_b_acc", kind="scatter")
plt.xlim(0, 100)

In [ ]:
# df[df["ids_to_load_train2"]]["ids_to_load_train2"]=1
df["ids_to_load_train2"] = df["ids_to_load_train2"].replace(0, 1)
df


In [ ]:
run.summary.keys()

In [ ]:
run_name

In [ ]:
ids_to_load_train